In [57]:
import sqlite3
from socket import *
import sys
import ssl
import pickle
from sqlalchemy import Column, Integer, String, UniqueConstraint, create_engine, exc, orm
from sqlalchemy.schema import Sequence
from sqlalchemy.ext.declarative import declarative_base
from contextlib import contextmanager

In [203]:
DBNAME = 'name_server'
folder = '/home/karimlulu/repos/NetworkProgramming/Name server'
DSN = 'sqlite:///{path}.db'.format(path=os.path.join(folder, DBNAME))

Base = declarative_base()

class Service(Base):
    __tablename__ = 'services'
    id = Column('id', Integer, primary_key=True)
    host = Column('host', String)
    port = Column('port', Integer)
    name = Column('name', String)
    description = Column('description', String)
    __table_args__ = (UniqueConstraint('host', 'port', name='address_cnt'),)
    
    def __repr__(self):
        return "<Service(name='{name}', address='{addr}', description='{desc}')>".format(name=self.name, 
                                                                                         addr=self.address,
                                                                                         desc=self.description) 
    def __str__(self):
        return repr(self)
    
    
    @property
    def address(self):
        return '{host}:{port}'.format(host=self.host, port=self.port)

    
class DatabaseAdapter(object):

    def __init__(self, dsn):
        try:
            eng = create_engine(dsn)
        except Exception as err:
            raise

        try:
            eng.connect()
        except:
            raise
        self.Session = orm.sessionmaker(bind=eng)
        self.services = Service.__table__
        self.eng = self.services.metadata.bind = eng
 

    @contextmanager
    def session_scope(self):
        session = self.Session()
        try:
            yield session
            session.commit()
        except Exception as error:
            session.rollback()
            raise
        finally:
            session.close()
    
    def upsert(self, obj):
        with self.session_scope() as ses:
            query = ses.query(Service).filter(Service.host==obj.host, Service.port==obj.port)
            if query.count():
                query.update({Service.name: obj.name, 
                              Service.description: obj.description or Service.description
                             })
            else:
                ses.add(obj)

                
    def get_service_by_name(self, name):
        with self.session_scope() as ses:
            query = ses.query(Service).filter(Service.name==name)
            total = query.count()
            if total > 1:
                msg = 'There are several services with the name `{name}`:\n'.format(name=name)
                for i,service in enumerate(query):
                    msg += '{idx}. {service}\n'.format(idx=i+1, service=service)
            elif total == 1:
                msg = 'Address of the requested service is {addr}'.format(addr=query.one().address)
            else:
                msg = 'Sorry, there is no service with the name `{name}`'.format(name=name)
            return msg 

In [204]:
db = DatabaseAdapter(DSN)

In [205]:
Base.metadata.create_all()

In [206]:
s = Service(host='127.0.0.1', port=80, name='a', description='create the space')
db.upsert(s)

In [211]:
s1 = Service(host='127.0.0.1', port=800, name='ab')
db.upsert(s1)

In [213]:
s2 = Service(host='127.0.0.1', port=800, name='a', description = 'Booms')
db.upsert(s2)

In [214]:
q = db.Session().query(Service).filter(Service.host=='127.0.0.1')
list(q)

[<Service(name='a', address='127.0.0.1:80', description='create the space')>,
 <Service(name='a', address='127.0.0.1:800', description='Booms')>]

In [217]:
m=db.get_service_by_name('a')
print(m)

There are several services with the name `a`:
1. <Service(name='a', address='127.0.0.1:80', description='create the space')>
2. <Service(name='a', address='127.0.0.1:800', description='Booms')>



In [125]:
conn = sqlite3.connect(os.path.join(folder, DBNAME)+'.db')
cur = conn.cursor()
cur.execute('SELECT* FROM services')
print(cur.fetchall())
conn.commit()

[(1, '127.0.0.1', 80, 'a')]
